In [9]:
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import nltk


# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')
# Download WordNet if not already downloaded
nltk.download('wordnet')
nltk.data.path.append("/path/to/nltk_data")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define the scope
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

# Authorize the client
client = gspread.authorize(creds)

# Open the Google Form responses spreadsheet
sheet = client.open('Your Google Form Responses').sheet1

# Get all values from the sheet
response_values = sheet.get_all_values()

# Assuming the first row contains headers, and the rest are responses
headers = response_values[0]
responses = response_values[1:]

# Convert responses to DataFrame
new_responses_df = pd.DataFrame(responses, columns=headers)

# Load existing dataset from CSV
existing_data = pd.read_csv('Student Mental Wellness Survey (Responses) - Form responses 1.csv')

# Combine existing dataset with new responses
data = pd.concat([existing_data, new_responses_df], ignore_index=True)




APIError: {'code': 403, 'message': 'Google Drive API has not been used in project 498572782644 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=498572782644 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'errors': [{'message': 'Google Drive API has not been used in project 498572782644 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=498572782644 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'domain': 'usageLimits', 'reason': 'accessNotConfigured', 'extendedHelp': 'https://console.developers.google.com'}], 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console API activation', 'url': 'h

In [ ]:
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
data

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Haryana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,21,Bihar,Male,Bachelor's 3rd Year,Btech cse,No,No,No change,No,No,No,No,No,No change,Na,No,No
119,21,Madhya Pradesh,Male,Bachelor's 4th Year,Computer Science and Communication Engineering,Sometimes,Sometimes,No change,No,No,Sometimes,No,No,Slight decline,NaN,I am not sure.,No
120,20,Haryana,Female,Bachelor's 3rd Year,Btech,Often,Often,No change,Sometimes,No,Often,No,Everyday,Slight decline,NaN,Yes,Yes
121,19,Assam,Male,Bachelor's 1st Year,Computer Science Engineering,Sometimes,Sometimes,No change,No,Rarely,Rarely,No,No,Slight decline,NaN,No,No


In [ ]:
data['X3'].unique()

array(['Odisha', 'Liquid', 'Haryana ', 'UP ', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh ', 'Uttar Pradesh ',
       'Maharashtra ', 'Bihar ', 'Uttar pradesh', 'West Bengal ',
       'Maharastra ', '-', 'Madhya Pradesh ', 'Jharkhand ', 'Tamil Nadu ',
       'UP', 'Madhya pradesh ', 'Uttar Pradesh', 'Haryana', 'Odisha ',
       'Jharkhand', 'Rajasthan', 'Chhattisgarh', 'HARYANA ',
       'Chhattisgarh ', 'MP', 'Karnataka ', 'Maharashtra',
       'Not from India', 'West Bengal', 'Gujrat ', 'Delhi NCR',
       'Madhya Pradesh', 'UttarPradesh', 'uttar Pradesh', 'Assam'],
      dtype=object)

In [ ]:
data = data[data['X3'] != 'Liquid']
data.loc[:, 'X3'] = data['X3'].str.strip().str.capitalize()
data['X3'].unique()

array(['Odisha', 'Haryana', 'Up', 'Delhi', 'Rajisthan', 'Bihar',
       'Uttar pradesh', 'Maharashtra', 'West bengal', 'Maharastra', '-',
       'Madhya pradesh', 'Jharkhand', 'Tamil nadu', 'Rajasthan',
       'Chhattisgarh', 'Mp', 'Karnataka', 'Not from india', 'Gujrat',
       'Delhi ncr', 'Uttarpradesh', 'Assam'], dtype=object)

In [ ]:
data.loc[:, 'X3'] = data['X3'].str.strip()
data.loc[:, 'X3'] = data['X3'].str.lower()
replacements = {'up': 'Uttar Pradesh', 'uttarpradesh': 'Uttar Pradesh', 'mp': 'Madhya Pradesh', 'delhi ncr': 'Delhi'}
data.loc[:, 'X3'] = data['X3'].replace(replacements)

data.loc[:, 'X3'] = data['X3'].apply(lambda x: x.title())

data['X3'].value_counts()

X3
Odisha            19
Madhya Pradesh    17
Uttar Pradesh     14
Jharkhand         14
Maharashtra       12
Bihar             12
Assam              7
Haryana            6
West Bengal        5
Delhi              5
Maharastra         2
Chhattisgarh       2
Rajisthan          1
Tamil Nadu         1
Rajasthan          1
Karnataka          1
Not From India     1
Gujrat             1
-                  1
Name: count, dtype: int64

In [ ]:
data['X16'] = data['X16'].astype(str)

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]


data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)

columns_to_encode = [col for col in data.columns if col not in ['X2', 'X6', 'X17', 'X16', 'X16_sentiment']]
data_encoded = pd.get_dummies(data, columns=columns_to_encode, drop_first=False)

data = data_encoded.drop(columns=['X16','X6'])
data


C:\Users\pande\AppData\Local\Temp\ipykernel_9928\39786436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['X16'] = data['X16'].astype(str)
C:\Users\pande\AppData\Local\Temp\ipykernel_9928\39786436.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) e

,X2,X17,X16_sentiment,X3_-,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Gujrat,X3_Haryana,...,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline,X18_No,X18_Yes
0,20,Yes,0.0,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
2,20,No,0.0,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,False,True
3,21,Yes,0.0,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
4,56,No,0.0,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,True,False
5,23,I am not sure.,0.0,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,21,No,0.0,False,False,True,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
119,21,I am not sure.,0.0,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,True,False
120,20,Yes,0.0,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,True,False,True
121,19,No,0.0,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,True,True,False


In [ ]:
data['X16_sentiment'].value_counts()

X16_sentiment
 0.0000    110
-0.4215      2
-0.7430      2
-0.5106      1
 0.6249      1
-0.8889      1
 0.5837      1
 0.3382      1
 0.4310      1
 0.8715      1
-0.2732      1
Name: count, dtype: int64

In [ ]:
# Save DataFrame to CSV file
data.to_csv('data_with_sentiment.csv', index=False)


In [ ]:
X = data.drop(columns=['X17'])  
y = data['X17']  

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X, y)

with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)